In [5]:
import sys
import os
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)

# Data manipulation and analysis
import pandas as pd
import numpy as np
import pickle 

import GEMS_TCO
from GEMS_TCO import kernels 
from GEMS_TCO import orderings as _orderings
from GEMS_TCO import load_data

import torch
from collections import defaultdict

import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import copy
from GEMS_TCO import configuration as config
from pathlib import Path

In [6]:
lat_lon_resolution = [20,20]
years = ['2024']
month_range =[7,8]
nheads = 200
mm_cond_number = 10 

data_load_instance = load_data(config.mac_data_load_path)
df = data_load_instance.read_pickle(config.mac_estimates_day_path,config.mac_full_day_v05_pickle)

df.head()

for day in range(12,13):
    print(f'\n Day {day} data size per day: { (200/lat_lon_resolution[0])*(100/lat_lon_resolution[0])  } \n')

    idx_for_datamap= [ 8*(day),8*(day+1)]

    params = list(df.iloc[day-1][:-1])
    params = torch.tensor(params, dtype=torch.float64, requires_grad=True)

    input_path = Path(config.mac_data_load_path)
   
    map, ord_mm, nns_map= data_load_instance.load_mm20k_data_bymonthyear( lat_lon_resolution= lat_lon_resolution, mm_cond_number=mm_cond_number,years_=years, months_=month_range)
    analysis_data_map, aggregated_data = data_load_instance.load_working_data_byday( map, ord_mm, nns_map, idx_for_datamap= idx_for_datamap)



 Day 12 data size per day: 50.0 



In [ ]:
result = {}

for day in range(12,13):
    print(f'\n Day {day} data size per day: { (200/lat_lon_resolution[0])*(100/lat_lon_resolution[0])  } \n')

    idx_for_datamap= [ 8*(day),8*(day+1)]

    params = list(df.iloc[day-1][:-1])
    params = torch.tensor(params, dtype=torch.float64, requires_grad=True)

    input_path = Path(config.mac_data_load_path)
   
    map, ord_mm, nns_map= data_load_instance.load_mm20k_data_bymonthyear( lat_lon_resolution= lat_lon_resolution, mm_cond_number=mm_cond_number,years_=years, months_=month_range)
    analysis_data_map, aggregated_data = data_load_instance.load_working_data_byday( map, ord_mm, nns_map, idx_for_datamap= idx_for_datamap)

    instance = kernels.model_fitting(
        smooth= 1.0,
        input_map=analysis_data_map,
        aggregated_data=aggregated_data,
        nns_map=nns_map,
        mm_cond_number=mm_cond_number,
        nheads= nheads
    )

    # optimizer = optim.Adam([params], lr=0.01)  # For Adam
    optimizer, scheduler = instance.optimizer_fun(params, lr=0.03, betas=(0.9, 0.99), eps=1e-8, step_size=100, gamma=0.9)    
    out = instance.run_full(params, optimizer,scheduler, instance.matern_cov_anisotropy_kv, epochs=500)
    result[day+1] = out
        

### Save estimates in to pickle fime

In [ ]:
isinstance(result, dict)
import os
# Save pickle
output_filename = f"estimation_1250_july24.pkl"

# base_path = "/home/jl2815/tco/data/pickle_data"
output_path = "/Users/joonwonlee/Documents/"
output_filepath = os.path.join(output_path, output_filename)
with open(output_filepath, 'wb') as pickle_file:
    pickle.dump(result, pickle_file)

input_filepath = output_filepath
# Load pickle
with open(input_filepath, 'rb') as pickle_file:
    loaded_map = pickle.load(pickle_file)

loaded_map

## load estimates from amarel and make it into pd data frame

10x10 full     takes 22 seconds (epochs 444)

vecchia log
10 by 10 fix 300 epochs (2m 7.2)  
original 2m 7.2  25 3 2.8 -0.04 -0.04 0.0066 4.3

if dont use cache   300 epochs fixed 5m 35.3
25 3 2.84 -0.04 -0.04 0.0066 4.3